In [1]:
# Affine/Softmax 계층 구현하기

# 신경망의 순전파 때, 가중치 신호의 총합을 계산하기 위해 행렬의 곱을 사용한다.
# 이 때 중요한 점은 행렬의 곱에서 대응하는 차원의 원소 수를 일치시키는 것이다.
# 신경망의 순전파 때 수행하는 행렬의 곱을 기하학에서는 어파인 변환Affine transformation이라고 하며, 이 책에서는 어파인 변환을 수행하는 처리를 'Affine 계층'으로 구현한다.

In [5]:
import numpy as np

# 순전파 때의 편향 덧셈은 X*W에 대한 편향이 각 데이터마다 더해진다. 예를 들어 N=2(데이터가 두 개)인 경우 편향은 그 두 데이터에 각각 더해진다.
X_dot_W = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])
print(X_dot_W)
print(X_dot_W+B)

# 편향의 역전파 때는 각 데이터의 역전파 값이 편향의 원소에 모여야한다.
dY = np.array([[1,2,3], [4,5,6]])
print(dY)
dB = np.sum(dY, axis=0) # 2개로 흩어져있던 역전파 값이 1개의 편향 원소로 모인다.
print(dB)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]
[[1 2 3]
 [4 5 6]]
[5 7 9]


In [6]:
# Affine 계층
class Affine:
    def __init__(self, W, b):
        self.W = W # 학습된 가중치 매개변수
        self.b = b # 학습된 편향 매개변수
        self.x = None # 입력 값
        self.dW = None # 가중치 매개변수의 기울기(미분 값)
        self.dB = None # 편향 매개변수의 기울기(미분 값)
    
    def forward(self, x): # 인자로 입력값을 받음
        self.x = x
        out = np.dot(x, self.W) + self.b # (X*W)+B
        
        return out
    
    def backward(self, dout): # 인자로 출력 미분값을 받음
        dx = np.dot(dout, self.W.T) # dout*(W의 전치행렬) --> 역전파 최종 반환값(입력값에 대한 역전파 값)
        self.dW = np.dot(self.x.T, dout) # (X의 전치행렬)*dout --> 가중치 매개변수 갱신
        self.dB = np.sum(dout, axis=0) # 역전파로 편향 모으기(sum) --> 편향 매개변수 갱신
        
        return dx

In [ ]:
# softmax 함수는 입력 값을 정규화하여 출력한다. 즉 출력 값들의 총합을 1로 만들어준다.
# 손실 함수인 교차 엔트로피 오차도 포함하여 Softmax-with-Loss 계층을 만들어보자.
'''
신경망에서 수행하는 작업은 '학습'과 '추론' 두 가지인데, 추론 할 때는 일반적으로 (마지막에) Softmax 계층을 사용하지 않는다.
또한, 신경망에서 정규화하지 않는 출력 결과를 점수score라고 한다.
신경망 추론에서 답을 하나만 내는 경우에는, 모든 출력 결과들 중 점수가 가장 높은 하나의 결과만 가지면 되므로 굳이 정규화하는 시간과 비용을 낭비할 필요가 없다.
반면, 신경망을 학습할 때는 Softmax 계층을 거쳐서 정규화 해 주어야 한다.
정규화 해 준 값을 교차 엔트로피 오차 손실 함수를 거치면서 정답 레이블과 비교 후 가중치 매개변수 갱신을 해야하기 때문이다.
'''
# 'softmax 함수'의 손실 함수로 '교차 엔트로피 오차 함수'를, '항등 함수'의 손실 함수로 '오차제곱합 함수'를 이용하면 역전파의 결과 (y-t) 형식으로 말끔히 떨어진다.

In [8]:
# Softmax-with-Loss 계층
# 

import sys, os
sys.path.append(os.pardir)
from common.functions import *


class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실
        self.y = None # softmax의 출력
        self.t = None # 정답 레이블(원-핫 벡터)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0] # 전파 값을 나눌 배치 크기
        dx = (self.y - self.t) / batch_size # 배치 사이즈로 나눔으로써 데이터 1개당 오차를 앞계층으로 전파한다
        
        return dx